<a href="https://colab.research.google.com/github/rickiepark/llm-from-scratch/blob/main/ch07/01_main-chapter-code/exercise-solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
세바스찬 라시카(Sebastian Raschka)가 쓴 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>의 번역서 예제 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rickiepark/llm-from-scratch">https://github.com/rickiepark/llm-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 7장 연습문제 솔루션


## 연습문제 7.1: 프롬프트 스타일 변경


다음과 같은 데이터 항목이 있다고 가정해 보겠습니다.

```json
{
  "instruction": "Identify the correct spelling of the following word.",
  "input": "Ocassion",
  "output": "The correct spelling is 'Occasion.'"
}
```

본문에서는 Alpaca 스타일 프롬프트 템플릿에 따라 다음과 같이 형식을 지정했습니다.

```
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Occassion

### Response:
The correct spelling is 'Occasion.'
```

이 연습문제에서는 Phi-3 프롬프트 템플릿을 사용합니다. 이 템플릿은 데이터 항목의 형식을 다음과 같이 지정합니다.

```
<user>
Identify the correct spelling of the following word: 'Occasion'

<assistant>
The correct spelling is 'Occasion'.
```

이 프롬프트 템플릿은 상당히 짧아 입력 프롬프트가 짧아지므로 LLM 미세 ㅌ닝 및 텍스트 생성을 위한 런타임 및 하드웨어 요구 사항이 줄어듭니다.
이 변경을 위해 `format_input` 함수를 다음과 같이 업데이트합니다.



In [1]:
def format_input(entry):
    instruction_text = (
        f"<|user|>\n{entry['instruction']}"
    )

    input_text = f"\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

두 개의 입력 샘플(하나는 `'input'` 필드에 내용이 있고 하나는 없는 샘플)에 적용하여 의도한 대로 작동하는지 확인해 보겠습니다.


In [2]:
sample_data = [
    {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"},
    {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}
]

print(format_input(sample_data[0]))
print()
print(format_input(sample_data[1]))

<|user|>
Identify the correct spelling of the following word.
Ocassion

<|user|>
What is an antonym of 'complicated'?


다음으로, 응답에 <|assistant|> 프롬프트 템플릿을 사용하도록 `InstructionDataset` 클래스도 업데이트합니다.


```python
import tiktoken
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        # 텍스트 토큰화
        self.encoded_texts = []
        for entry in data:

            ###################################################################
            # 추가: `format_input_phi` 사용 및 응답 텍스트 템플릿 조정
            instruction_plus_input = format_input(entry)
            response_text = f"\n<|assistant|>:\n{entry['output']}"
            ###################################################################
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

    def __getitem__(self, index):
        return self.encoded_texts[index]

    def __len__(self):
        return len(self.data)


tokenizer = tiktoken.get_encoding("gpt2")
```


마지막으로 테스트 세트 응답을 수집할 때 생성된 응답을 추출하는 방식도 업데이트해야 합니다.


```python
for i, entry in tqdm(enumerate(test_data), total=len(test_data)):

    input_text = format_input(entry)
    tokenizer=tokenizer

    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)

    # 추가: ###Response -> <|assistant|> 로 조정
    response_text = generated_text[len(input_text):].replace("<|assistant|>:", "").strip()

    test_data[i]["model_response"] = response_text
```


편의상 연습문제 솔루션은 [exercise_experiments.py](exercise_experiments.py) 스크립트에 구현되어 있으며, 다음과 같이 실행할 수 있습니다:


```bash
python exercise_experiments.py --exercise_solution phi3_prompt
```

출력:

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 935
Validation set length: 55
Test set length: 110
--------------------------------------------------
Device: cuda
--------------------------------------------------
...
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Initial losses
   Training loss: 3.71630220413208
   Validation loss: 3.6440994262695314
Ep 1 (Step 000000): Train loss 2.633, Val loss 2.622
...
Ep 2 (Step 000230): Train loss 0.424, Val loss 0.928
<|user|> Convert the active sentence to passive: 'The chef cooks the meal every day.' <|assistant|>: The meal is prepared every day by the chef....
Training completed in 1.50 minutes.
Plot saved as loss-plot-phi3-prompt.pdf
--------------------------------------------------
Generating responses
100% 110/110 [00:11<00:00,  9.27it/s]
Responses saved as instruction-data-with-response-phi3-prompt.json
Model saved as gpt2-medium355M-sft-phi3-prompt.pth
```

비교를 위해, 원래 7장의 미세 튜닝 코드를 `python exercise_experiments.py --exercise_solution baseline`을 통해 실행할 수 있습니다.

Nvidia L4 GPU에서 위의 코드는 Phi-3 프롬프트 템플릿을 사용하여 실행하는 데 1.5분이 걸립니다. 반면 Alpaca 스타일 템플릿은 실행하는 데 1.80분이 걸립니다. Phi-3 템플릿이 더 짧은 모델 입력을 생성하므로 약 17% 더 빠릅니다.

응답이 올바르게 포맷팅되었는지 확인하기 위해 몇 가지 응답을 살펴보겠습니다.

```json
    {
        "instruction": "Rewrite the sentence using a simile.",
        "input": "The car is very fast.",
        "output": "The car is as fast as lightning.",
        "model_response": "The car is as fast as a cheetah."
    },
    {
        "instruction": "What type of cloud is typically associated with thunderstorms?",
        "input": "",
        "output": "The type of cloud typically associated with thunderstorms is cumulonimbus.",
        "model_response": "The type of cloud associated with thunderstorms is a cumulus cloud."
    },
    {
        "instruction": "Name the author of 'Pride and Prejudice'.",
        "input": "",
        "output": "Jane Austen.",
        "model_response": "The author of 'Pride and Prejudice' is Jane Austen."
    },
```

Ollama Llama 3 방법을 사용하여 성능을 평가할 수 있습니다. 이 방법은 편의를 위해 `python exercise_experiments.py` 스크립트에도 구현되어 있으며 다음과 같이 실행할 수 있습니다.

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-phi3-prompt.json
```

출력:

```
Ollama running: True
Scoring entries: 100%|████████████████████████| 110/110 [01:08<00:00,  1.60it/s]
Number of scores: 110 of 110
Average score: 48.87
```

점수는 50에 가깝습니다. 이는 이전에 Alpaca 스타일 프롬프트로 달성한 점수와 비슷합니다.

Phi 프롬프트 스타일이 더 나은 본질적인 이점이나 근거는 없지만, 아래 *팁* 섹션에서 언급한 주의 사항을 제외하고는 더 간결하고 효율적일 수 있습니다.
```

#### 팁: 특수 토큰 고려하기


- Phi-3 프롬프트 템플릿에는 `<|user|>` 및 `<|assistant|>`와 같은 특수 토큰이 포함되어 있는데, 이는 GPT-2 토크나이저에 최적이 아닐 수 있습니다.
- GPT-2 토크나이저는 `<|endoftext|>`를 특수 토큰(토큰 ID 50256으로 인코딩됨)으로 인식하지만, 앞서 언급한 것과 같은 다른 특수 토큰을 처리하는 데는 비효율적입니다.
- 예를 들어, `<|user|>`는 5개의 개별 토큰 ID(27, 91, 7220, 91, 29)로 인코딩되는데, 이는 매우 비효율적입니다.
- `tiktoken`에서 `allowed_special` 인수를 통해 `<|user|>`를 새로운 특수 토큰으로 추가할 수 있지만, GPT-2 어휘사전을 수정하지 않고는 이를 처리할 수 없다는 점을 유의하세요.
- 토크나이저와 LLM이 특수 토큰을 처리하도록 확장하는 방법에 대해 궁금하다면, [extend-tiktoken.ipynb](../../ch05/09_extending-tokenizers/extend-tiktoken.ipynb) 보너스 자료를 참조하세요. (여기서는 필요하지 않지만, 호기심 많은 독자를 위한 내용입니다.)
- 또한, 어휘사전을 통해 프롬프트 템플릿의 특수 토큰을 지원하는 모델은 더 효율적이고 전반적으로 더 나은 성능을 보일 수 있다고 가정할 수 있습니다.


&nbsp;
## 연습문제 7.2: 명령어 및 입력 마스킹

다음 그림과 같이 명령어를 마스킹하려면 `InstructionDataset` 클래스와 `custom_collate_fn`을 약간 수정해야 합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/mask-instructions.webp" width=800px>



In [4]:
# 이 `format_input` 함수는 원래 7장 코드에서 복사되었습니다.
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

`InstructionDataset` 클래스를 수정하여 지시의 길이를 수집할 수 있습니다. 이 길이는 콜레이트 함수를 코딩할 때 타깃에서 지시 내용 위치를 찾는 데 사용됩니다. 다음과 같습니다.


In [5]:
import torch
from torch.utils.data import Dataset


class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        ##########################################################################################
        # 추가: 지시 길이를 위한 별도의 리스트
        self.instruction_lengths = []
        ##########################################################################################

        self.encoded_texts = []

        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text

            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

            ##########################################################################################
            # 추가: 지시 길이 수집
            instruction_length = len(tokenizer.encode(instruction_plus_input))
            self.instruction_lengths.append(instruction_length)
            ##########################################################################################

    def __getitem__(self, index):
        # 추가: 지시 길이와 텍스트를 모두 따로 반환
        return self.instruction_lengths[index], self.encoded_texts[index]

    def __len__(self):
        return len(self.data)

In [6]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

다음으로, `InstructionDataset` 데이터셋의 변경 사항으로 인해 각 `batch`가 이제 `item` 대신 `(instruction_length, item)` 튜플을 포함하도록 `custom_collate_fn`을 업데이트합니다. 또한 타깃 ID 리스트에서 지시 토큰을 마스킹합니다.


In [7]:
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
    allowed_max_length=None,
    device="cpu"
):
    # 배치에서 가장 긴 시퀀스 찾기
    batch_max_length = max(len(item)+1 for instruction_length, item in batch)   # 추가: batch는 이제 튜플입니다.

    # 입력과 타깃을 패딩하고 준비
    inputs_lst, targets_lst = [], []

    for instruction_length, item in batch:  # 추가: batch는 이제 튜플입니다.
        new_item = item.copy()
        # <|endoftext|> 토큰 추가
        new_item += [pad_token_id]
        # 시퀀스를 max_length까지 패딩
        padded = new_item + [pad_token_id] * (batch_max_length - len(new_item))
        inputs = torch.tensor(padded[:-1])  # 입력을 위해 마지막 토큰 자르기
        targets = torch.tensor(padded[1:])  # 타깃을 위해 오른쪽으로 +1 이동

        # 타깃에서 첫 번째를 제외한 모든 패딩 토큰을 ignore_index로 바꾸기
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index

        ##########################################################################################
        # 추가: 타깃에서 모든 입력 및 지시 토큰 마스킹
        targets[:instruction_length-1] = -100
        ##########################################################################################

        # 선택적으로 최대 시퀀스 길이로 자르기
        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]

        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # 입력 및 타깃 리스트를 텐서로 변환하고 타깃 장치로 전송
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor

아래 샘플 데이터로 시험해 보겠습니다.


In [8]:
sample_data = [
    {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."},
    {'instruction': 'Sort the following list in alphabetical order.', 'input': 'Zebra, Elephant, Crocodile', 'output': 'Crocodile, Elephant, Zebra'},
    {'instruction': 'Arrange the given numbers in descending order.', 'input': '5, 12, 8, 3, 15', 'output': '15, 12, 8, 5, 3.'}
]

In [9]:
from torch.utils.data import DataLoader

train_dataset = InstructionDataset(sample_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=len(sample_data),
    collate_fn=custom_collate_fn,
    num_workers=0
)

In [10]:
print("훈련 데이터 로더:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

훈련 데이터 로더:
torch.Size([3, 64]) torch.Size([3, 64])


In [11]:
print("입력:\n", inputs[1])
print("\n\n타깃:\n", targets[1])

입력:
 tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198, 42758,   262,  1708,  1351,   287, 24830,
          605,  1502,    13,   198,   198, 21017, 23412,    25,   198,    57,
        37052,    11, 42651,    11,  9325, 19815,   576,   198,   198, 21017,
        18261,    25,   198,    34, 12204,   375,   576,    11, 42651,    11,
         1168, 37052, 50256, 50256])


타깃:
 tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,   198,   198, 21017, 18261,
           25,   198,    34, 12204,   375,   576,    11, 42651,    11,  1168,
        37052, 

`targets` 텐서를 기반으로 볼 수 있듯이, 지시와 패딩 토큰 모두 이제 -100 플레이스홀더 토큰을 사용하여 마스킹되었습니다.
입력이 올바른지 확인하기 위해 디코딩해 보겠습니다.


In [12]:
print(tokenizer.decode(list(inputs[1])))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Sort the following list in alphabetical order.

### Input:
Zebra, Elephant, Crocodile

### Response:
Crocodile, Elephant, Zebra<|endoftext|><|endoftext|>


다음으로, 마스킹되지 않은 타깃 토큰 ID를 디코딩해 보겠습니다:


In [13]:
non_masked_targets = targets[1][targets[1] != -100]

print(tokenizer.decode(list(non_masked_targets)))



### Response:
Crocodile, Elephant, Zebra<|endoftext|>


위에서 보여준 것처럼 마스킹되지 않은 타깃 토큰에는 의도한 대로 `"Instruction"` 및 `"Input"` 필드가 제외됩니다. 이제 수정된 코드를 실행하여 이 마스킹 전략을 사용하여 미세 조정할 때 LLM이 얼마나 잘 수행되는지 확인할 수 있습니다.

편의를 위해 다음과 같이 `exercise_experiments.py` 코드를 사용하여 비교를 실행할 수 있습니다.


```bash
python exercise_experiments.py --exercise_solution mask_instructions
```

출력:

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 935
Validation set length: 55
Test set length: 110
--------------------------------------------------
Device: cuda
--------------------------------------------------
...
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Initial losses
   Training loss: 2.280539035797119
   Validation loss: 2.262560224533081
Ep 1 (Step 000000): Train loss 1.636, Val loss 1.620
...
Ep 2 (Step 000230): Train loss 0.143, Val loss 0.727
...
Training completed in 1.77 minutes.
Plot saved as loss-plot-mask-instructions.pdf
--------------------------------------------------
Generating responses
100% 110/110 [02:10<00:00,  1.19s/it]
Responses saved as instruction-data-with-response-mask-instructions.json
Model saved as gpt2-medium355M-sft-mask-instructions.pth
```

다음으로, 결과 LLM의 성능을 평가해 보겠습니다.

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-mask-instructions.json
```

```
Ollama running: True
Scoring entries: 100%|██████████████████████████████████████████████████████████████████████████████████████| 110/110 [01:23<00:00,  1.31it/s]
Number of scores: 110 of 110
Average score: 47.73
```

점수를 기반으로 볼 때, 지시 마스킹은 "Instruction Tuning With Loss Over Instructions" 논문(https://arxiv.org/abs/2405.14394)에서 관찰된 것과 일치하게 약간 더 나쁜 성능을 보입니다.


&nbsp;
## 연습문제 7.3: 원본 Alpaca 데이터셋에서 파인튜닝


Stanford Alpaca 데이터셋([https://github.com/tatsu-lab/stanford_alpaca](https://github.com/tatsu-lab/stanford_alpaca))에서 모델을 미세 튜닝하려면 파일 URL을 다음에서

```python
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch07/01_main-chapter-code/instruction-data.json"
```

다음으로 변경하기만 하면 됩니다.

```python
url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json"
```

이 데이터셋에는 52,000개의 항목(7장에서 사용한 것보다 50배 더 많음)이 포함되어 있으며 각 항목의 길이는 7장에서 사용했던 것보다 더 깁니다.
따라서 GPU에서 훈련을 실행하는 것이 좋습니다.

메모리 부족 오류가 발생하면 배치 크기를 8에서 4, 2 또는 1로 줄이는 것을 고려하세요. 배치 크기를 줄이는 것 외에도 `allowed_max_length`를 1024에서 512 또는 256으로 줄이는 것을 고려할 수 있습니다.


편의를 위해 `exercise_experiments.py` 코드를 사용하여 52k Alpaca 데이터셋에서 배치 크기 4와 `allowed_max_length` 512로 모델을 미세 조정할 수 있습니다. 다음과 같습니다.


```bash
python exercise_experiments.py --exercise_solution alpaca_52k
```

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 44201
Validation set length: 2601
Test set length: 5200
--------------------------------------------------
Device: cuda
--------------------------------------------------
...
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Initial losses
   Training loss: 3.3681655883789063
   Validation loss: 3.4122894287109373
Ep 1 (Step 000000): Train loss 2.477, Val loss 2.750
...
Ep 2 (Step 022095): Train loss 0.761, Val loss 1.557
...
Training completed in 196.38 minutes.
Plot saved as loss-plot-alpaca52k.pdf
--------------------------------------------------
Generating responses
100% 5200/5200 [2:56:33<00:00,  2.04s/it]
Responses saved as instruction-data-with-response-alpaca52k.json
Model saved as gpt2-medium355M-sft-alpaca52k.pth
```


아래는 Alpaca 데이터셋의 샘플 몇 개와 생성된 모델 응답입니다.


```json
[
    {
        "instruction": "Edit the following sentence to increase readability: \"He made a huge effort and was so successful.\"",
        "input": "",
        "output": "He exerted a tremendous effort, and thus enjoyed great success.",
        "model_response": "He put in an immense effort and was rewarded with success."
    },
    {
        "instruction": "Rewrite the following sentence to make it more concise: \"I was displeased with the result of the experiment that I conducted.\"",
        "input": "",
        "output": "I was unhappy with my experiment's outcome.",
        "model_response": "I was displeased with the results of the experiment."
    },
    {
        "instruction": "How can we build a more efficient GPT model?",
        "input": "",
        "output": "We can build a more efficient GPT model by optimizing the architecture of the model, using smaller model sizes and training with fewer parameters. We can also leverage techniques such as knowledge distillation, transfer learning, dynamic sparsity and hybrid computing to further improve the efficiency of the model.",
        "model_response": "Building a more efficient GPT model requires careful planning and optimization. First, it is important to identify the target language and the context in which the model is used. Then, it is important to select the appropriate model architecture, such as backpropagation, hyperparameters, and hyperparameters. Finally, it is important to select the appropriate model weights and optimizers, such as backpropagation, hyperparameters, and hyperparameters."
    },
]


마지막으로, `ollama_evaluate.py` 유틸리티를 사용하여 미세 튜닝된 LLM을 평가할 수 있습니다.

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-alpaca52k.json
```

```
Scoring entries: 100%|████████████████████| 5200/5200 [1:07:52<00:00, 1.28it/s]
Number of scores: 5188 of 5200
Average score: 48.16
```


이 점수는 이 장에서 사용했던 데이터셋에서 얻은 점수보다 약간 낮습니다. 그러나 Alpaca 테스트 세트에는 본문에서 사용했던 데이터셋보다 더 다양하고 부분적으로 더 어려운 지시가 포함되어 있습니다.


## 연습문제 7.4: LoRA를 사용한 파라미터 효율적인 미세 튜닝


LoRA를 사용하여 모델을 지시 미세 튜닝하려면 부록 E의 관련 클래스 및 함수를 사용하세요.

```python
from appendix_E import LoRALayer, LinearWithLoRA, replace_linear_with_lora
```


다음으로, 7.5절의 모델 로딩 코드 아래에 다음 코드 라인들을 추가합니다:


```python
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters before: {total_params:,}")

for param in model.parameters():
    param.requires_grad = False

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters after: {total_params:,}")
replace_linear_with_lora(model, rank=16, alpha=16)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"훈련 가능한 총 LoRA 파라미터 개수: {total_params:,}")
model.to(device)
```


편의를 위해 `exercise_experiments.py` 코드를 사용하여 rank 16과 alpha 16인 LoRA를 사용해 모델을 미세 튜닝할 수 있습니다. 다음과 같습니다.


```bash
python exercise_experiments.py --exercise_solution lora
```

Output:

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 935
Validation set length: 55
Test set length: 110
--------------------------------------------------
Device: cuda
--------------------------------------------------
File already exists and is up-to-date: gpt2/355M/checkpoint
File already exists and is up-to-date: gpt2/355M/encoder.json
File already exists and is up-to-date: gpt2/355M/hparams.json
File already exists and is up-to-date: gpt2/355M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/355M/model.ckpt.index
File already exists and is up-to-date: gpt2/355M/model.ckpt.meta
File already exists and is up-to-date: gpt2/355M/vocab.bpe
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Total trainable parameters before: 406,286,336
Total trainable parameters after: 0
Total trainable LoRA parameters: 7,898,384
Initial losses
   Training loss: 3.7684114456176756
   Validation loss: 3.7619335651397705
Ep 1 (Step 000000): Train loss 2.509, Val loss 2.519
...
Ep 2 (Step 000230): Train loss 0.308, Val loss 0.652
...
--------------------------------------------------
Generating responses
100% 110/110 [01:52<00:00,  1.03s/it]
Responses saved as instruction-data-with-response-lora.json
Model saved as gpt2-medium355M-sft-lora.pth
```

비교를 위해 `python exercise_experiments.py --exercise_solution baseline` 명령으로 원래 7장의 미세 튜닝된 코드를 실행할 수 있습니다.

Nvidia L4 GPU에서 LoRA를 사용하여 위 코드를 실행하면 1.3분이 걸리니다. 기본 모델은 1.8분이 걸립니다. 따라서 LoRA가 약 28% 빠릅니다.

Ollama Llama 3 모델을 사용해 성능을 평가할 수 있습니다. 편의를 위해
`python exercise_experiments.py`에 구현되어 있으며 다음처럼 실행할 수 있습니다.

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-lora.json
```

출력:

```
Ollama running: True
Scoring entries: 100%|████████████████████████| 110/110 [01:13<00:00,  1.50it/s]
Number of scores: 110 of 110
Average score: 50.23
```

원래 모델과 비슷한 50점 정도의 점수입니다.
